In [1]:
# install beautifulsoup for the first time if not there
#!conda install BeautifulSoup4
#!pip install BeautifulSoup4

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
html="""<html>
    <head>
        <title>
            This is an example page
        </title>
    </head>
    <body>
        Here is the content of the page
        <p>
            One paragraph
        </p>
        <li> one </li>
        <li> two </li>
        <table>
            <tr>
                <td>col 1</td>
                <td>col 2</td>
            </tr>
            <tr>
                <td>val 1</td>
                <td>val 2</td>
            </tr>
        </table>
        End of body
    </body>
</html>"""

In [3]:
soup = BeautifulSoup(html, 'html')

In [8]:
def print_html_tree(tag,level = 0):
    print('\t'*level + tag.name)
    for child in tag.findChildren():
        print_html_tree(child, level+1)

In [12]:
for tag in soup.find_all('td'):
    print(tag.text)

col 1
col 2
val 1
val 2


In [10]:
#print_html_tree(soup.currentTag)

Extract names and titles of [MIS Faulty at UGA](https://www.terry.uga.edu/directory/mis/)

How many assistant professors are there?

In [22]:
r = requests.get("https://www.terry.uga.edu/directory/mis")

In [23]:
r.status_code

200

In [24]:
html = r.text

In [25]:
type(html)

str

In [47]:
soup = BeautifulSoup(html, 'html.parser')

In [28]:
names = soup.find_all('td', id="name")

In [33]:
titles = soup.find_all('td', id="title")

In [48]:
emails = soup.find_all('td', id="email")

In [50]:
emails[0].text

'dgoodhue@uga.edu'

In [38]:
titles[0].text

'Professor Emeritus '

In [37]:
names[0].text

'Dale Louis Goodhue'

In [39]:
len(names) == len(titles)

True

In [40]:
len(names)

33

In [51]:
# assemble the data
res = []
for i in range(len(names)):
    entry = {'Name': names[i].text.strip(), 'Title': titles[i].text.strip(), 'Email': emails[i].text.strip()}
    res.append(entry)
    

In [43]:
len(res)

33

In [52]:
data = pd.DataFrame(res)

In [53]:
data.head()

,Email,Name,Title
0,dgoodhue@uga.edu,Dale Louis Goodhue,Professor Emeritus
1,KADIANNE.FRANCIS@uga.edu,Kadianne Francis,Academic Advisor II
2,weifeng.li@uga.edu,Weifeng Li,Assistant Professor
3,accidentalcio@uga.edu,Timothy (Tim) Chester,Adjunct Instructor
4,rwatson@uga.edu,Richard (Rick) Thomas Watson,J. Rex Fuqua Distinguished Chair for Internet ...


In [56]:
max_len = max(data.Email.str.len())

In [57]:
max_len

24

In [60]:
data[data.Email.str.len() == max_len]

,Email,Name,Title
1,KADIANNE.FRANCIS@uga.edu,Kadianne Francis,Academic Advisor II


In [46]:
sum(data.Title == 'Assistant Professor')

6

## Exercise

Extract the names, parties and states of [the US Senate](https://www.congress.gov/members?pageSize=100&q={%22congress%22:%22116%22,%22chamber%22:%22Senate%22})

What are the states that have R and D senators?

In [68]:
r = requests.get('https://www.congress.gov/members?pageSize=100&q=%7B%22congress%22:%22116%22,%22chamber%22:%22Senate%22%7D')

In [69]:
r.status_code

200

In [70]:
html = r.text

In [71]:
soup = BeautifulSoup(html, 'html')

In [72]:
# extract members data
members = soup.find_all('li', class_="expanded")

In [95]:
res = []
for m in members:
    # extract the name
    name = m.find("span", class_="result-heading").text    
    # the remaining items are
    remaining_items = m.find_all("span", class_="result-item")
    # extract state and party
    state = remaining_items[0].find("span").text
    party = remaining_items[1].find("span").text
    res.append({'Name': name, 'State': state, 'Party': party})

In [98]:
data=pd.DataFrame(res)

In [100]:
len(data)

100

In [101]:
data.head()

,Name,Party,State
0,"Senator Alexander, Lamar",Republican,Tennessee
1,"Senator Baldwin, Tammy",Democratic,Wisconsin
2,"Senator Barrasso, John",Republican,Wyoming
3,"Senator Bennet, Michael F.",Democratic,Colorado
4,"Senator Blackburn, Marsha",Republican,Tennessee


# what are the states that have rep. from both parties?

In [104]:
res1 = data.groupby(["State", "Party"]).Name.count()

In [107]:
res2 = res1.unstack('Party')

In [120]:
res2[res2.Democratic==res2.Republican]

Party,Democratic,Independent,Republican
State,,,
Alabama,1.0,NaN,1.0
Arizona,1.0,NaN,1.0
Colorado,1.0,NaN,1.0
Montana,1.0,NaN,1.0
Ohio,1.0,NaN,1.0
Pennsylvania,1.0,NaN,1.0
West Virginia,1.0,NaN,1.0
Wisconsin,1.0,NaN,1.0


In [122]:
res2[res2.Independent > 0]

Party,Democratic,Independent,Republican
State,,,
Maine,NaN,1.0,1.0
Vermont,1.0,1.0,NaN
